In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import ExponentialSmoothing
import warnings 
warnings.filterwarnings("ignore")

In [41]:
sales = pd.read_csv(r'C:\Users\ADMIN\Downloads\Codes\TimeSeries Demand Classification & Forecasting\Demand Classification\Monthly Frequency\Imputed Sales Monthly.csv')
daily_classification_data = pd.read_csv(r'C:\Users\ADMIN\Downloads\Codes\TimeSeries Demand Classification & Forecasting\Demand Classification\Daily Frequency\Forecast Classification Daily.csv')
weekly_classification_data = pd.read_csv(r'C:\Users\ADMIN\Downloads\Codes\TimeSeries Demand Classification & Forecasting\Demand Classification\Weekly Frequency\Forecast Classification Weekly.csv')
monthly_classification_data = pd.read_csv(r'C:\Users\ADMIN\Downloads\Codes\TimeSeries Demand Classification & Forecasting\Demand Classification\Monthly Frequency\Forecast Classification Monthly.csv')                                        

In [42]:
output = pd.DataFrame()
def halts_winter(material_id):
    global sales
    data = sales.loc[sales['StockCode']==material_id]
    if len(data)>=12:
        data = data.drop(['Unnamed: 0','StockCode'], axis=1)
        data = data.set_index('InvoiceDate')
        # Create an ETS model
        model = ExponentialSmoothing(data, trend='add', seasonal='add', seasonal_periods=4)
        # Fit the model
        fit = model.fit()
        # Forecast for the testing set
        forecast = fit.forecast(12)
        forecast_df = pd.DataFrame(forecast, columns=['Forecast'])
        forecast_df['StockCode']=material_id
        forecast_df = forecast_df.reset_index()
        forecast_df = forecast_df.rename(columns={'index': 'InvoiceDate'})
        forecast_df['Forecast'] = round(forecast_df['Forecast'])
        global output
        output = output.append(forecast_df,ignore_index=True)

In [43]:
daily_smooth_list = list(daily_classification_data[daily_classification_data['Class']=='Smooth demand']['StockCode'])
weekly_smooth_list = list(weekly_classification_data[weekly_classification_data['Class']=='Smooth demand']['StockCode'])

In [44]:
for i in daily_smooth_list:
    halts_winter(i)

In [47]:
output = output.pivot_table(index='InvoiceDate', columns='StockCode', values='Forecast')
output.to_csv(r'C:\Users\ADMIN\Downloads\Codes\TimeSeries Demand Classification & Forecasting\Forecasting\halts-winter.csv')